In [ ]:
import os
from glob import glob

import psutil

import numpy as np
import matplotlib.pyplot as plt

from decode_trf import decode_trf
from mosaiq_field_export import DeliveryData

In [ ]:
config = {
    "linac_logfile_data_directory": "S:\\Physics\\Programming\\data\\LinacLogFiles",
    "machine_types": {
        "elekta-agility": {
            "max_leaf_gap": 400,
            "leaf_pair_widths": [
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5
            ]
        }
    }    
}

grid_resolution = 0.5  # mm
machine_type = 'elekta-agility'

max_leaf_gap = config['machine_types'][machine_type]['max_leaf_gap']
leaf_pair_widths = np.array(config['machine_types'][machine_type]['leaf_pair_widths'])

In [ ]:
Y1_LEAF_BANK_NAMES = [
    'Y1 Leaf {}/Scaled Actual (mm)'.format(item)
    for item in range(1, 81)
]

Y2_LEAF_BANK_NAMES = [
    'Y2 Leaf {}/Scaled Actual (mm)'.format(item)
    for item in range(1, 81)
]

JAW_NAMES = [
    'X1 Diaphragm/Scaled Actual (mm)', 'X2 Diaphragm/Scaled Actual (mm)']

GANTRY_NAME = 'Step Gantry/Scaled Actual (deg)'
COLLIMATOR_NAME = 'Step Collimator/Scaled Actual (deg)'


def _pull_single_logfile(logfile_path):
    logfile_dataframe = decode_trf(logfile_path)
    raw_monitor_units = logfile_dataframe[
        'Step Dose/Actual Value (Mu)'].values.tolist()

    diff = np.append([0], np.diff(raw_monitor_units))
    diff[diff < 0] = 0

    monitor_units = np.cumsum(diff).tolist()

    gantry = logfile_dataframe[GANTRY_NAME].values.tolist()
    collimator = logfile_dataframe[COLLIMATOR_NAME].values.tolist()

    y1_bank = [
        logfile_dataframe[name].values.tolist()
        for name in Y1_LEAF_BANK_NAMES
    ]

    y2_bank = [
        logfile_dataframe[name].values.tolist()
        for name in Y2_LEAF_BANK_NAMES
    ]

    mlc = [y1_bank, y2_bank]

    jaw = [
        logfile_dataframe[name].values.tolist()
        for name in JAW_NAMES
    ]

    logfile_delivery_data = DeliveryData(
        monitor_units, gantry, collimator, mlc, jaw
    )

    return logfile_delivery_data

In [ ]:
test_files = glob(os.path.join(
    config["linac_logfile_data_directory"],
    'indexed', '*', '012194*', 'clinical', '*_VMAT', '*', '*.trf'
))

delivery_data = _pull_single_logfile(test_files[0])

In [ ]:
def calc_a_single_blocked_fraction(start_diffs, end_diffs,
                                   start_blocked, end_blocked):    
    blocked_fraction = np.ones(np.shape(start_diffs)) * np.nan
    all_open = ~start_blocked & ~end_blocked
    blocked_fraction[all_open] = 0

    all_blocked = start_blocked & end_blocked
    blocked_fraction[all_blocked] = 1

    start_blocked_fraction = np.copy(blocked_fraction)
    end_blocked_fraction = np.copy(blocked_fraction)
    
    partial_blocked = start_blocked != end_blocked
    travel = np.abs(
        start_diffs[partial_blocked] - 
        end_diffs[partial_blocked])
    
    start_partial_blocked_ref = start_blocked[partial_blocked]
    end_partial_blocked_ref = end_blocked[partial_blocked]
    
    start_blocked_fraction[partial_blocked & start_blocked] = np.abs(
        start_diffs[partial_blocked][start_partial_blocked_ref] / 
        travel[start_partial_blocked_ref]
    )
    start_blocked_fraction[partial_blocked & end_blocked] = 0
    
    end_blocked_fraction[partial_blocked & end_blocked] = np.abs(
        end_diffs[partial_blocked][end_partial_blocked_ref] / 
        travel[end_partial_blocked_ref]
    )
    end_blocked_fraction[partial_blocked & start_blocked] = 0
    
    assert np.all(~np.isnan(start_blocked_fraction))
    assert np.all(~np.isnan(end_blocked_fraction))
    
    return start_blocked_fraction, end_blocked_fraction


def calc_leaf_blocked_fractions(leaf_xx, mlc):
    start_left_diffs = leaf_xx - -mlc[0:-1,:,0][:,:,None]
    end_left_diffs = leaf_xx - -mlc[1::,:,0][:,:,None]
    
    start_left_blocked = start_left_diffs <= 0
    end_left_blocked = end_left_diffs <= 0
    
    (
        start_left_blocked_fraction, end_left_blocked_fraction
    ) = calc_a_single_blocked_fraction(
        start_left_diffs, end_left_diffs,
        start_left_blocked, end_left_blocked)

    start_right_diffs = leaf_xx - mlc[0:-1,:,1][:,:,None]
    end_right_diffs = leaf_xx - mlc[1::,:,1][:,:,None]
    
    start_right_blocked = start_right_diffs >= 0
    end_right_blocked = end_right_diffs >= 0
    
    (
        start_right_blocked_fraction, end_right_blocked_fraction
    ) = calc_a_single_blocked_fraction(
        start_right_diffs, end_right_diffs,
        start_right_blocked, end_right_blocked)
    
    return {
        'start_left_blocked_fraction': start_left_blocked_fraction,
        'end_left_blocked_fraction': end_left_blocked_fraction,
        'start_right_blocked_fraction': start_right_blocked_fraction,
        'end_right_blocked_fraction': end_right_blocked_fraction
    }


def calc_jaw_blocked_fraction(grid_y, jaw, repeats):
    start_top_diffs = grid_y - jaw[0:-1,1][:,None]
    end_top_diffs = grid_y - jaw[1::,1][:,None]
    
    start_top_blocked = start_top_diffs >= 0
    end_top_blocked = end_top_diffs >= 0
    
    (
        start_top_blocked_fraction, end_top_blocked_fraction
    ) = calc_a_single_blocked_fraction(
        start_top_diffs, end_top_diffs,
        start_top_blocked, end_top_blocked)
    
    start_bottom_diffs = grid_y - -jaw[0:-1,0][:,None]
    end_bottom_diffs = grid_y - -jaw[1::,0][:,None]
    
    start_bottom_blocked = start_bottom_diffs <= 0
    end_bottom_blocked = end_bottom_diffs <= 0
    
    (
        start_bottom_blocked_fraction, end_bottom_blocked_fraction
    ) = calc_a_single_blocked_fraction(
        start_bottom_diffs, end_bottom_diffs,
        start_bottom_blocked, end_bottom_blocked)
    
    return {
        'start_top_blocked_fraction': np.repeat(
            start_top_blocked_fraction[:,:,None], repeats, axis=2),
        'end_top_blocked_fraction': np.repeat(
            end_top_blocked_fraction[:,:,None], repeats, axis=2),
        'start_bottom_blocked_fraction': np.repeat(
            start_bottom_blocked_fraction[:,:,None], repeats, axis=2),
        'end_bottom_blocked_fraction': np.repeat(
            end_bottom_blocked_fraction[:,:,None], repeats, axis=2)
    }

In [ ]:
def determine_leaves_to_be_calced(grid_yy, grid_leaf_map, jaw):
    calc_top = np.invert(grid_yy[:,0] >= np.max(jaw[:,1]))
    top_leaves_calc = np.unique(grid_leaf_map[calc_top])

    calc_bot = np.invert(grid_yy[:,0] <= np.min(-jaw[:,0]))
    bot_leaves_calc = np.unique(grid_leaf_map[calc_bot])

    leaves_to_be_calced = np.intersect1d(top_leaves_calc, bot_leaves_calc)
    
    return leaves_to_be_calced

In [ ]:
def determine_x_pos_to_be_calced(leaf_xx, mlc):
    leaf_x = leaf_xx[0,:]
    calc_left = np.invert(leaf_x <= np.min(-mlc[:,:,0]))
    calc_right = np.invert(leaf_x >= np.max(mlc[:,:,1]))

    x_pos_to_be_calced = calc_left & calc_right
    
    return x_pos_to_be_calced

In [ ]:
def calc_blocked_fraction(leaf_xx, mlc, grid_leaf_map, 
                          grid_yy, jaw):
    
    virtual_memory_start = psutil.virtual_memory()
    
    leaves_to_be_calced = determine_leaves_to_be_calced(
        grid_yy, grid_leaf_map, jaw)
    x_pos_to_be_calced = determine_x_pos_to_be_calced(leaf_xx, mlc)
    
    subset_leaf_xx = []
    
    leaf_blocked_fractions = calc_leaf_blocked_fraction_define_subset(
        leaf_xx, grid_yy, mlc, grid_leaf_map)

    jaw_blocked_fractions = calc_jaw_blocked_fraction(
        grid_yy[:,0], jaw, len(leaf_xx[0,:]))
    
    all_start_blocked_fractions = np.concatenate([
        np.expand_dims(leaf_blocked_fractions['start_left_blocked_fraction'], axis=0),
        np.expand_dims(leaf_blocked_fractions['start_right_blocked_fraction'], axis=0),
        np.expand_dims(jaw_blocked_fractions['start_top_blocked_fraction'], axis=0),
        np.expand_dims(jaw_blocked_fractions['start_bottom_blocked_fraction'], axis=0)
    ], axis=0)

    start_blocked_fraction = np.max(all_start_blocked_fractions, axis=0)

    all_end_blocked_fractions = np.concatenate([
        np.expand_dims(leaf_blocked_fractions['end_left_blocked_fraction'], axis=0),
        np.expand_dims(leaf_blocked_fractions['end_right_blocked_fraction'], axis=0),
        np.expand_dims(jaw_blocked_fractions['end_top_blocked_fraction'], axis=0),
        np.expand_dims(jaw_blocked_fractions['end_bottom_blocked_fraction'], axis=0)
    ], axis=0)

    end_blocked_fraction = np.max(all_end_blocked_fractions, axis=0)

    blocked_fraction = start_blocked_fraction + end_blocked_fraction
    blocked_fraction[blocked_fraction > 1] = 1
    
    virtual_memory_end = psutil.virtual_memory()
    ram_used = virtual_memory_end.used - virtual_memory_start.used
    
    return blocked_fraction, ram_used


In [ ]:
def calc_mu_density_over_slice(mu, mlc, jaw, slice_to_check,
                               grid_xx, grid_yy, leaf_xx, grid_leaf_map):

    blocked_fraction, ram_used = calc_blocked_fraction(
        leaf_xx, mlc[slice_to_check,:,:], grid_leaf_map, 
        grid_yy, jaw[slice_to_check,:])

    mu_density = np.sum(np.diff(mu[slice_to_check])[:,None,None] * (1 - blocked_fraction), axis=0)
    
    return mu_density, ram_used

In [ ]:
def calc_max_index(current_index, number_of_sections, final_index):
    max_index = current_index + number_of_sections + 1
    if max_index > final_index:
        max_index = final_index
        
    return max_index


def calc_mu_density(delivery_data, grid_resolution, 
                    max_leaf_gap, leaf_pair_widths, 
                    ram_fraction=0.8, **kwargs):
    min_number_of_sections = 20
    
    number_of_sections = min_number_of_sections
    current_index = 0
    
    mu = np.array(delivery_data.monitor_units)
    mu_diff = np.diff(mu)
    no_change = mu_diff == 0
    no_change_before = no_change[0:-1]
    no_change_after = no_change[1::]

    no_change_before_and_after = no_change_before & no_change_after
    irrelevant_control_point = np.hstack([no_change[0], no_change_before_and_after, no_change[-1]])
    control_points_to_use = np.invert(irrelevant_control_point)
    
    mlc = np.swapaxes(delivery_data.mlc, 0, 2)
    jaw = np.swapaxes(delivery_data.jaw, 0, 1)
    
    mu = mu[control_points_to_use]
    mlc = mlc[control_points_to_use,:,:]
    jaw = jaw[control_points_to_use,:]
    
    min_x = np.floor(np.min(-mlc[:,:,0]) / grid_resolution) * grid_resolution
    max_x = np.ceil(np.max(mlc[:,:,1]) / grid_resolution) * grid_resolution

    min_y = np.min(-jaw[:,0])
    max_y = np.max(jaw[:,1])

    leaf_x = np.arange(
        min_x, max_x + grid_resolution, grid_resolution
    ).astype('float')

    total_leaf_widths = np.sum(leaf_pair_widths)
    leaf_y = np.cumsum(leaf_pair_widths) - leaf_pair_widths/2 - total_leaf_widths/2

    leaf_xx, leaf_yy = np.meshgrid(leaf_x, leaf_y)

    initial_leaf_grid_y_pos = leaf_y[len(leaf_y)//2]

    top_grid_pos = (
        (max_y - initial_leaf_grid_y_pos) // grid_resolution *
        grid_resolution + initial_leaf_grid_y_pos)

    bot_grid_pos = (
        initial_leaf_grid_y_pos -
        (-min_y + initial_leaf_grid_y_pos) // grid_resolution *
        grid_resolution)

    grid_y = np.arange(
        bot_grid_pos, top_grid_pos + grid_resolution, grid_resolution)

    grid_leaf_map = np.argmin(np.abs(grid_y[:,None] - leaf_y[None,:]), axis=1)

    grid_xx, grid_yy = np.meshgrid(leaf_x, grid_y)
    adjusted_grid_leaf_map = grid_leaf_map - np.min(grid_leaf_map)

    leaves_to_be_calced = np.unique(grid_leaf_map)
    adjusted_mlc = mlc[:,leaves_to_be_calced,:]
    adjusted_leaf_xx = leaf_xx[leaves_to_be_calced,:]

    final_index = len(mu)
    max_index = 0
    
    mu_density = np.zeros_like(grid_xx)
    
    while max_index < final_index:
        max_index = calc_max_index(current_index, number_of_sections, final_index)
        
        slice_to_check = slice(current_index, max_index, 1)
        mu_density_of_slice, ram_used = calc_mu_density_over_slice(
            mu, adjusted_mlc, jaw, slice_to_check, 
            grid_xx, grid_yy, adjusted_leaf_xx, adjusted_grid_leaf_map)
        mu_density += mu_density_of_slice
        
        current_index = current_index + number_of_sections
        
        current_ram_fraction = ram_used / psutil.virtual_memory().available
        if current_ram_fraction != 0:
            number_of_sections = number_of_sections * int(
                np.floor(ram_fraction / current_ram_fraction))
            
            if number_of_sections < min_number_of_sections:
                number_of_sections = min_number_of_sections
        
    return grid_xx, grid_yy, mu_density

In [ ]:
def calc_leaf_blocked_fraction_define_subset(leaf_xx, grid_yy, mlc, grid_leaf_map):
    leaf_blocked_fractions = calc_leaf_blocked_fractions(leaf_xx, mlc)

    for key in leaf_blocked_fractions.keys():
        leaf_blocked_fractions[key] = leaf_blocked_fractions[key][:, grid_leaf_map, :]
        
    return leaf_blocked_fractions

In [ ]:
grid_xx, grid_yy, mu_density = calc_mu_density(
    delivery_data, grid_resolution, max_leaf_gap, 
    leaf_pair_widths, ram_fraction=0.95)

In [ ]:
# np.where(mu_density2 - mu_density > 0.0001)

In [ ]:
plt.pcolormesh(
    grid_xx, 
    grid_yy,
    mu_density)
plt.colorbar()

In [ ]:
# mu = np.array(delivery_data.monitor_units)

In [ ]:
# mu_diff = np.diff(mu)
# no_change = mu_diff == 0
# no_change_before = no_change[0:-1]
# no_change_after = no_change[1::]

# no_change_before_and_after = no_change_before & no_change_after

In [ ]:
# irrelevant_control_point = np.hstack([no_change[0], no_change_before_and_after, no_change[-1]])
# np.where(irrelevant_control_point)